# Benin (Malanville) — EDA
This notebook performs profiling, cleaning and exploratory data analysis for the Benin (Malanville) dataset.

Instructions: Update `DATA_PATH` if needed to point at `data/benin-malanville.csv`. Do NOT commit any generated CSVs.

In [ ]:
# Configuration
DATA_PATH = '../data/benin-malanville.csv'  # relative to notebook file; adjust if needed
CLEAN_OUT_PATH = '../data/benin_clean.csv'  # exported but do NOT commit
SAMPLE_ROWS = None  # set to an int to run on a sample for fast iteration

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

In [ ]:
# Load data
df = pd.read_csv(DATA_PATH, parse_dates=[0], nrows=SAMPLE_ROWS)
# normalize column names (strip spaces/newlines)
df.columns = df.columns.str.strip().str.replace(' ', '').str.replace('
','')
df.rename(columns={df.columns[0]: 'Timestamp'}, inplace=True)
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
df = df.set_index('Timestamp')
df.head()

## Summary statistics & missing values

In [ ]:
# Describe numeric columns
display(df.describe())
# Missing value counts and percent
missing_counts = df.isna().sum()
missing_pct = (missing_counts / len(df)) * 100
missing_report = pd.DataFrame({'missing_count': missing_counts, 'missing_pct': missing_pct})
display(missing_report.sort_values('missing_pct', ascending=False))
# Columns with >5% nulls
cols_gt5pct = missing_report[missing_report['missing_pct'] > 5].index.tolist()
print('Columns with >5% nulls:', cols_gt5pct)

## Outlier detection (Z-score) & basic cleaning

In [ ]:
from scipy.stats import zscore
cols_for_z = ['GHI','DNI','DHI','ModA','ModB','WS','WSgust']
# make sure columns exist after name normalization
cols_for_z = [c for c in cols_for_z if c in df.columns]
z = df[cols_for_z].apply(lambda x: zscore(x, nan_policy='omit'))
outlier_mask = (z.abs() > 3)
# Flag rows with any outlier in these columns
df['outlier_flag'] = outlier_mask.any(axis=1)
print('Outlier counts per column:')
print(outlier_mask.sum())

In [ ]:
# Impute median for key columns (GHI, DNI, DHI, ModA, ModB, WS, WSgust)
key_cols = ['GHI','DNI','DHI','ModA','ModB','WS','WSgust']
key_cols = [c for c in key_cols if c in df.columns]
for c in key_cols:
    median = df[c].median()
    df[c] = df[c].fillna(median)
# Add a cleaning flag: rows changed by imputation or outlier trimming?
df['cleaning_flag'] = df['outlier_flag'] | df[key_cols].isna().any(axis=1)
df['cleaning_flag'] = df['cleaning_flag'].astype(int)
df.head()

## Time series plots (GHI, DNI, DHI, Tamb)

In [ ]:
plot_cols = [c for c in ['GHI','DNI','DHI','Tamb'] if c in df.columns]
fig, axes = plt.subplots(len(plot_cols), 1, figsize=(12, 3*len(plot_cols)), sharex=True)
if len(plot_cols) == 1:
    axes = [axes]
for ax, col in zip(axes, plot_cols):
    df[col].plot(ax=ax, title=col)
plt.tight_layout()
plt.show()

## Cleaning impact: ModA & ModB pre/post-clean

In [ ]:
for mod in ['ModA','ModB']:
    if mod in df.columns:
        display(df.groupby('cleaning_flag')[mod].mean())

## Correlation heatmap and scatter plots

In [ ]:
corr_cols = [c for c in ['GHI','DNI','DHI','TModA','TModB','ModA','ModB','Tamb','RH','WS','WSgust'] if c in df.columns]
if corr_cols:
    plt.figure(figsize=(10,8))
    sns.heatmap(df[corr_cols].corr(), annot=True, fmt='.2f', cmap='coolwarm')
    plt.title('Correlation heatmap')
    plt.show()
# Scatter examples
for x in ['WS','WSgust','WD']:
    if x in df.columns and 'GHI' in df.columns:
        plt.figure(figsize=(6,4))
        sns.scatterplot(data=df, x=x, y='GHI', alpha=0.3)
        plt.title(f'GHI vs {x}')
        plt.show()

## Wind & distribution analysis

In [ ]:
# Histogram for GHI and WS
for c in ['GHI','WS']:
    if c in df.columns:
        plt.figure(figsize=(6,3))
        sns.histplot(df[c].dropna(), kde=True, bins=50)
        plt.title(c)
        plt.show()
# Simple wind rose-like polar plot using WD and WS (binned)
if 'WD' in df.columns and 'WS' in df.columns:
    wd = df['WD'].dropna()
    ws = df['WS'].dropna()
    # create bins
    bins = np.linspace(0, 360, 13)
    inds = np.digitize(wd % 360, bins)
    rose = [ws[inds==i].mean() for i in range(1,len(bins))]
    angles = np.deg2rad((bins[:-1] + bins[1:]) / 2)
    plt.figure(figsize=(6,6))
    ax = plt.subplot(111, polar=True)
    ax.bar(angles, rose, width=np.diff(np.deg2rad(bins)), align='center', alpha=0.7)
    plt.title('Wind rose (mean WS by WD bin)')
    plt.show()

## Temperature & humidity analysis; Bubble chart

In [ ]:
# RH vs Tamb and RH vs GHI
if 'RH' in df.columns and 'Tamb' in df.columns:
    plt.figure(figsize=(6,4))
    sns.scatterplot(data=df.sample(min(5000, len(df))), x='Tamb', y='RH', alpha=0.3)
    plt.title('RH vs Tamb')
    plt.show()
if 'RH' in df.columns and 'GHI' in df.columns:
    plt.figure(figsize=(6,4))
    sns.scatterplot(data=df.sample(min(5000, len(df))), x='GHI', y='RH', alpha=0.3)
    plt.title('RH vs GHI')
    plt.show()
# Bubble chart: GHI vs Tamb, bubble size = RH
if 'GHI' in df.columns and 'Tamb' in df.columns and 'RH' in df.columns:
    plt.figure(figsize=(8,6))
    sample = df.sample(min(2000, len(df)))
    sizes = (sample['RH'].fillna(sample['RH'].median()) - sample['RH'].min() + 1) * 5
    plt.scatter(sample['Tamb'], sample['GHI'], s=sizes, alpha=0.5, c=sample['RH'], cmap='viridis')
    plt.xlabel('Tamb')
    plt.ylabel('GHI')
    plt.title('GHI vs Tamb (bubble size ~ RH)')
    plt.colorbar(label='RH')
    plt.show()

In [ ]:
# Export cleaned CSV (do NOT commit data/ to git)
df.to_csv(CLEAN_OUT_PATH, index=True)
print('Wrote cleaned CSV to', CLEAN_OUT_PATH)